In [1]:
import sys
sys.path.append('..')

In [2]:
from utils import load_model_from_path

path = "/nlp/scr3/nlp/llms-in-llms/mrt5/models/vowel_removal/MrT5/simple_vowel_removal_pctrl20%_seed85_seed85/checkpoints/checkpoint-10000"
model = load_model_from_path("MrT5", model_path=path)

/nlp/scr/kallini/miniconda3/envs/charlm-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path: /nlp/scr3/nlp/llms-in-llms/mrt5/models/vowel_removal/MrT5/simple_vowel_removal_pctrl20%_seed85_seed85/checkpoints/checkpoint-10000


In [3]:
model.config

T5Config {
  "_name_or_path": "/nlp/scr3/nlp/llms-in-llms/mrt5/models/vowel_removal/MrT5/simple_vowel_removal_pctrl20%_seed85_seed85/checkpoints/checkpoint-10000",
  "architectures": [
    "MrT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "delete_gate_layer": 2,
  "deletion_threshold": -15.0,
  "deletion_type": "scaled_sigmoid",
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "eval_language": "en",
  "feed_forward_proj": "gated-gelu",
  "fixed_deletion_amount": 0.5,
  "gate_layer_norm": true,
  "gradient_checkpointing": false,
  "has_absolute_position_embeddings": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_heads": 4,
  "num_decoder_layers": 3,
  "num_heads": 6,
  "num_layers": 3,
  "pad_token_id": 0,
  "random_deletion_probability": 0.5,
  "relative_attention_m

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

/nlp/scr/kallini/miniconda3/envs/charlm-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
encoded = tokenizer("akjefjklsdafka", return_tensors="pt")

In [5]:
encoded

{'input_ids': tensor([[100, 110, 109, 104, 105, 109, 110, 111, 118, 103, 100, 105, 110, 100,
           1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
output_ids = model(input_ids=encoded["input_ids"], decoder_input_ids=encoded["input_ids"], output_attentions=True)

In [7]:
mean_norms = []
for k, q in zip(output_ids.cross_attention_keys, output_ids.cross_attention_queries):
    mean_norms.append(q.norm(dim=-1).mean() + k.norm(dim=-1).mean())
for k, q in zip(output_ids.encoder_keys[2:], output_ids.encoder_queries[2:]):
    mean_norms.append(q.norm(dim=-1).mean() + k.norm(dim=-1).mean())

In [8]:
sum(mean_norms) / len(mean_norms)

tensor(21.6549, grad_fn=<DivBackward0>)

In [33]:
output_ids.decoder_scores[0].norm(dim=-1).mean()

tensor(inf, grad_fn=<MeanBackward0>)

In [9]:
model.config

T5Config {
  "_name_or_path": "/nlp/scr3/nlp/llms-in-llms/mrt5/models/contextual_vowel_removal/MrT5/test_contextual_randomk_gumbel_seed68/checkpoints/checkpoint-30000",
  "architectures": [
    "MrT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "delete_gate_layer": 2,
  "deletion_threshold": -15.0,
  "deletion_top_k": 5,
  "deletion_type": "scaled_sigmoid",
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "eval_language": "en",
  "feed_forward_proj": "gated-gelu",
  "fixed_deletion_amount": 0.5,
  "gate_layer_norm": true,
  "gradient_checkpointing": false,
  "has_absolute_position_embeddings": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 3,
  "num_heads": 6,
  "num_layers": 3,
  "pad_token_id": 0,
  "random_deletion_probability": 0.5,
  "relative

In [3]:
import json
from tqdm import tqdm

def count_languages(json_file):
    language_count = {}

    with open(json_file, 'r') as file:
        # Create a progress bar for the file with `tqdm`
        for line in tqdm(file, desc="Processing lines"):
            try:
                data = json.loads(line)
                # Check if the 'language' key exists in the JSON object
                if 'language' in data:
                    language = data['language']
                    if language in language_count:
                        language_count[language] += 1
                    else:
                        language_count[language] = 1
            except json.JSONDecodeError:
                print("Error decoding JSON on this line:", line)

    return language_count

# Usage example, replace 'your_file.json' with your actual JSON file path
file_path = '/nlp/scr3/nlp/llms-in-llms/mrt5/lm_datasets/mc4-multilingual-train.json'
result = count_languages(file_path)
print("Language counts:", result)


Processing lines: 10240000it [19:02, 8961.93it/s] 

Language counts: {'fr': 683730, 'zh': 682190, 'vi': 682180, 'ru': 684169, 'hi': 682658, 'th': 682419, 'de': 683293, 'tr': 683178, 'ur': 682582, 'bg': 681281, 'es': 682210, 'sw': 681677, 'ar': 683507, 'en': 682630, 'el': 682296}
